In [0]:
%run ./01_Project_config

Load the bronze table to dataframe for cleaning and creating silver table

In [0]:
from pyspark.sql.functions import col, when, coalesce, lit, current_timestamp
from delta.tables import DeltaTable

# Load Bronze Transactions
bronze_transactions = spark.read.table(f"{catalog}.{bronze_schema}.card_transactions")

In [0]:
%sql
select * 
from fraud_lakehouse.bronze.card_transactions
where transaction_amount < 0;

In [0]:
%sql
select customer_id
from fraud_lakehouse.bronze.card_transactions
where customer_id is null;

In [0]:
%sql
select transaction_id, count(transaction_id) as total_trans_id
from fraud_lakehouse.bronze.card_transactions
group by transaction_id
having total_trans_id > 1;

In [0]:
%sql
select transaction_id 
from fraud_lakehouse.bronze.card_transactions
where transaction_id is null;

In [0]:
from pyspark.sql.functions import *
audit_df = bronze_transactions.filter(col("transaction_id").isNull())
display(audit_df)

adding null transaction_id inside audit table

In [0]:

quarantine_table = f"{catalog}.quarantine.audit_transactions"


audit_df.write.format("delta").mode("append").saveAsTable(quarantine_table)

spark.read.table("fraud_lakehouse.quarantine.audit_transactions").display()

Adding null customer id in another audit table audit_transaction1

In [0]:

audit_df1 = bronze_transactions.filter(col("customer_id").isNull())
display(audit_df1)

In [0]:
audit_df1.write.format("delta").mode("append").saveAsTable("fraud_lakehouse.quarantine.audit_transactions") 
spark.read.table("fraud_lakehouse.quarantine.audit_transactions").display()

In [0]:
%sql
select customer_id from fraud_lakehouse.bronze.card_transactions where customer_id is null;


In [0]:
from pyspark.sql import SparkSession

df = spark.table("fraud_lakehouse.bronze.card_transactions").write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("fraud_lakehouse.silver.silver_card_transactions")

In [0]:
%sql
select * from fraud_lakehouse.silver.silver_card_transactions;

creating audit tables in silver table by replacing transaction_is with "UNKNOWN" and customer_id with "ANON"

In [0]:
df = spark.read.table("fraud_lakehouse.silver.silver_card_transactions")
df_clean = df.filter(col("transaction_id").isNotNull() & col("customer_id").isNotNull() & col("merchant_id").isNotNull())
df_clean.display()

In [0]:
df_clean.write.format("delta").mode("overwrite").saveAsTable("fraud_lakehouse.silver.silver_card_transactions")

In [0]:
%sql
select * from fraud_lakehouse.silver.silver_card_transactions;

Reading the References files

In [0]:

(
    spark.read
        .format("csv")
        .option("header", "true")
        .option("inferSchema", "true")
        .load("/Volumes/fraud_lakehouse/raw_landing/references/customer_master_new.csv")
        .write
        .format("delta")
        .mode("overwrite")
        .saveAsTable("fraud_lakehouse.silver.dim_customer_master")
)


In [0]:
%python
from pyspark.sql.functions import col

df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Volumes/fraud_lakehouse/raw_landing/references/fx_rates_new.csv") \
    .withColumnRenamed("currency", "cur")

df.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("fraud_lakehouse.silver.dim_fx_rates")

In [0]:
(
    spark.read
        .format("csv")
        .option("header", "true")
        .option("inferSchema", "true")
        .load("/Volumes/fraud_lakehouse/raw_landing/references/merchant_blacklist_new.csv")
        .write
        .format("delta")
        .mode("overwrite")
        .saveAsTable("fraud_lakehouse.silver.dim_merchant_blacklist")
)


In [0]:

(
    spark.read
        .format("csv")
        .option("header", "true")
        .option("inferSchema", "true")
        .load("/Volumes/fraud_lakehouse/raw_landing/references/merchant_master_new.csv")
        .write
        .format("delta")
        .mode("overwrite")
        .saveAsTable("fraud_lakehouse.silver.dim_merchant_master")
)


performing join operations to join dimension table with silver to:
- fill null countries
- fill null currencies
- add usd transaction column after converting to usd
- fraud_flag with 1 for customer_id with reason of being blacklist else 0
- adding is_suspicious column according to blacklist dimension table

filling null countries using customer_master_dim

In [0]:
from pyspark.sql.functions import *

silver_df = spark.table("fraud_lakehouse.silver.silver_card_transactions")
customer_df = spark.table("fraud_lakehouse.silver.dim_customer_master").select("customer_id", "home_country")

joined_df = silver_df.join(customer_df, on="customer_id", how="left")
filled_df = joined_df.withColumn(
    "country",
    coalesce(silver_df["country"], customer_df["home_country"])
).drop(customer_df["home_country"])

filled_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("fraud_lakehouse.silver.silver_card_transactions")

In [0]:
%sql
select country from fraud_lakehouse.silver.silver_card_transactions where country is null;

filling null currencies using dimension table fx_rates_dim

In [0]:
from pyspark.sql.functions import *
df = spark.read.table("fraud_lakehouse.silver.dim_fx_rates")
df1 = df.withColumnRenamed("currency", "cur")
df1.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("fraud_lakehouse.silver.dim_fx_rates")


silver = spark.table("fraud_lakehouse.silver.silver_card_transactions")
fx = spark.table("fraud_lakehouse.silver.dim_fx_rates").select("country", "cur")

joined= silver.join(fx, on="country", how="left")
filled = joined.withColumn(
    "currency",
    coalesce(silver["currency"], fx["cur"])
).drop(fx["cur"])

filled.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("fraud_lakehouse.silver.silver_card_transactions")


In [0]:
%sql
select currency from fraud_lakehouse.silver.silver_card_transactions where currency is null;

CONVERTING CURRENCIES INTO USD RATES USING FX_RATES_DIM TABLE

In [0]:
from pyspark.sql.functions import *

Silver = spark.table("fraud_lakehouse.silver.silver_card_transactions")
Silver.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("fraud_lakehouse.silver.silver_card_transactions")


fx_df = spark.table("fraud_lakehouse.silver.dim_fx_rates").select("country", "usd_rate")


silver_copy_df = spark.table("fraud_lakehouse.silver.silver_card_transactions")
joined_df = silver_copy_df.join(fx_df, on="country", how="left")


converted_df = joined_df.withColumn(
    "transaction_amount_usd",
    col("transaction_amount") * col("usd_rate")
)

display(converted_df)

fraud_flag column with value 1 for when customer_id is blacklisted and has a reason, else 0

In [0]:
df_blacklist = spark.table("fraud_lakehouse.silver.dim_merchant_blacklist")
df_transactions = spark.table("fraud_lakehouse.silver.silver_card_transactions")

df_merged = df_blacklist.join(
    df_transactions,
    "merchant_id"
)

df_merged.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("fraud_lakehouse.silver.merchant_blacklist_transactions")

In [0]:
%sql
update fraud_lakehouse.silver.silver_card_transactions_copy set fraud_flag = 1 where transaction_id in (select transaction_id from fraud_lakehouse.silver.merchant_blacklist_transactions)

In [0]:
from pyspark.sql.functions import col

df = spark.table("fraud_lakehouse.silver.silver_card_transactions")
count_df = df.filter(col("fraud_flag") == 1).count()
display(spark.createDataFrame([(count_df,)], ["fraud_flag_1_count"]))

%md
APPLYING IQR LOGIC FOR CREATIHNG AN UPPER BOUND LIMIT OF TRANSACTIONS

In [0]:
%python
from pyspark.sql.functions import col, when, lit, expr

# Use converted_df from cell 38 instead of reading from table
curated = converted_df

# Only rows with USD amount
non_null_df = curated.filter(col("transaction_amount_usd").isNotNull())

# Q1 & Q3 per card_type
iqr_stats_df = (
    non_null_df
        .groupBy("card_type")
        .agg(
            expr("percentile_approx(transaction_amount_usd, 0.25)").alias("q1"),
            expr("percentile_approx(transaction_amount_usd, 0.75)").alias("q3")
        )
)

# Upper bound only
iqr_bounds_df = (
    iqr_stats_df
        .withColumn("iqr", col("q3") - col("q1"))
        .withColumn("upper_bound", col("q3") + 1.5 * col("iqr"))
)

# Join bounds
curated_with_bounds = curated.join(
    iqr_bounds_df,
    on="card_type",
    how="left"
)

# Flag ONLY upper bound violation
curated_final_df = curated_with_bounds.withColumn(
    "is_suspicious",
    when(
        col("transaction_amount_usd").isNull(), lit(0)
    ).when(
        col("transaction_amount_usd") > col("upper_bound"),
        lit(1)
    ).otherwise(lit(0))
)

# Cleanup
curated_final_df = curated_final_df.drop(
    "q1", "q3", "iqr", "upper_bound"
)

In [0]:
curated_final_df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("fraud_lakehouse.silver.silver_card_transactions")

spark.read.table("fraud_lakehouse.silver.silver_card_transactions").display()

In [0]:
%sql
ALTER TABLE fraud_lakehouse.silver.silver_card_transactions SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');
ALTER TABLE fraud_lakehouse.silver.silver_card_transactions DROP COLUMN _rescued_data;

In [0]:
%sql
select * from fraud_lakehouse.silver.silver_card_transactions
